<img align="Left" alt="Dados_ao_Cubo" src="../shared_content/DADOS-AO-CUBO-Versão-04-1.png" />
<img align="right" alt="MLNet" src="../shared_content/EJ3KsfJWsAEtDaZ.png" />

## Davi Ramos -> Cientista de Dados 👋
(davi.info@gmail.com)

[![Linkedin Badge](https://img.shields.io/badge/-LinkedIn-blue?style=flat-square&logo=Linkedin&logoColor=white&link=https://www.linkedin.com/in/davi-ramos/)](https://www.linkedin.com/in/davi-ramos/)
[![Twitter Badge](https://img.shields.io/badge/-Twitter-1DA1F2?style=flat-square&logo=Twitter&logoColor=white&link=https://twitter.com/Daviinfo/)](https://twitter.com/Daviinfo/)
<a href="https://github.com/DaviRamos"><img src="https://img.shields.io/github/followers/DaviRamos.svg?label=GitHub&style=social" alt="GitHub"></a>

# ML.Net - Auto ML

## Este exemplo irá demontrar o Uso de AutoML com ML.Net

In [1]:
// Instalar os Pacotes do Nuget

// ML.NET
#r "nuget:Microsoft.ML"  

// ML.NET AutoML
#r "nuget:Microsoft.ML.AutoML" 

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package Microsoft.ML version 1.5.2

Installed package Microsoft.ML.AutoML version 0.17.2

In [2]:
using Microsoft.ML;
using Microsoft.ML.AutoML;
using Microsoft.ML.Data;
using System;
using System.IO;

In [3]:
// Caminho dos arquivos
private static string TRAIN_PATH = "./house_train.csv";
private static string TEST_PATH = "./house_test.csv";
private static string VALIDATION_PATH = "./house_validate.csv";
private static string LABEL_NAME = "median_house_value";

In [4]:
public static void PrintRSquared(RegressionMetrics metrics)
{
    Console.WriteLine($"R^2: {metrics.RSquared}");
    Console.Write(Environment.NewLine);
}

In [6]:
            var context = new MLContext();

            // Text loader from scratch
            //var textLoaderOptions = new TextLoader.Options
            //{
            //    Separators = new[] {','},
            //    HasHeader = true,
            //    Columns = new[]
            //    {
            //        new TextLoader.Column("Features", DataKind.Single, 0, 7),
            //        new TextLoader.Column("Label", DataKind.Single, 8),
            //        new TextLoader.Column("OceanProximity", DataKind.String, 9)
            //    }
            //};

            //var loader = context.Data.CreateTextLoader(textLoaderOptions);
            //var loaderData = loader.Load(TRAIN_PATH);

            // Text loader from inference
            var inference = context.Auto().InferColumns(TRAIN_PATH, hasHeader: true, separatorChar: ',', labelColumnIndex: 8);

            var loader = context.Data.CreateTextLoader(inference.TextLoaderOptions);

            var trainData = loader.Load(TRAIN_PATH);
            var validationData = loader.Load(VALIDATION_PATH);
            var testData = loader.Load(TEST_PATH);

            var experimentSettings = new RegressionExperimentSettings
            {
                MaxExperimentTimeInSeconds = 60,
                OptimizingMetric = RegressionMetric.RSquared
            };

            // First training (on train data)
            var experimentResult = context.Auto()
                .CreateRegressionExperiment(experimentSettings)
                .Execute(trainData, validationData, labelColumnName: LABEL_NAME);
            
            var firstPredictions = experimentResult.BestRun.Model.Transform(testData);
            var firstMetrics = context.Regression.Evaluate(firstPredictions, LABEL_NAME);
            PrintRSquared(firstMetrics);

            // Second training (on train + validation data)
            var trainPlusValidationData = loader.Load(new MultiFileSource(TRAIN_PATH, VALIDATION_PATH));
            var refitModel = experimentResult.BestRun.Estimator.Fit(trainPlusValidationData);
            IDataView predictionsRefitOnTrainPlusValidation = refitModel.Transform(testData);
            var secondMetrics = context.Regression.Evaluate(predictionsRefitOnTrainPlusValidation, LABEL_NAME);
            PrintRSquared(secondMetrics);

            // Third training (on train + test + validation data)
            var trainPlusValidationPlusTestDataView = loader.Load(new MultiFileSource(TRAIN_PATH, VALIDATION_PATH, TEST_PATH));
            var refitModelOnValidationData = experimentResult.BestRun.Estimator.Fit(trainPlusValidationPlusTestDataView);

            context.Model.Save(refitModelOnValidationData, trainData.Schema, "./house_model.zip");